# Test plane-based registration
Test different plane-based registration methods

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.registration import get_correspondences, extract_corresponding_features
from planeslam.geometry.util import quat_to_rot_mat

%load_ext autoreload
%autoreload 2

### Load AirSim data

In [2]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [3]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

### Decoupled

In [4]:
from planeslam.registration import decoupled_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

[[ 4.17561936e+01]
 [-6.91389945e-02]
 [ 2.35085002e+02]]
[[-4.93224409e+01]
 [ 6.85646618e-14]
 [ 5.25042913e+00]]
[[-73.93652591]
 [ -0.26256389]
 [  8.19982805]]
[[ 3.96851642]
 [ 0.02358479]
 [-0.21498566]]
[[ 3.98560873e+00]
 [-1.96732949e-03]
 [-1.28063227e-01]]
[[ 2.30098658]
 [-0.00383921]
 [-0.02968871]]
[[ 4.49636042]
 [-0.01993057]
 [-0.03203159]]
[[ 4.86525903]
 [ 0.00808046]
 [-0.05318259]]
[[ 4.38436669e+00]
 [-3.92347594e-03]
 [-7.51966504e-03]]
[[ 4.28062767e+00]
 [-2.01415903e-01]
 [ 8.76014594e-04]]
[[ 4.51646742]
 [ 0.14195673]
 [-0.08669257]]
[[ 1.12253149e+01]
 [-8.96304618e-03]
 [-8.63307625e+00]]
[[-12.44248494]
 [ -0.17425084]
 [ 45.92897225]]
[[1.22454902e+03]
 [3.76937261e+00]
 [5.35459938e+03]]
[[-0.56798381]
 [ 0.06901303]
 [-0.10303344]]
[[-3.28092888e+01]
 [-1.45730019e-02]
 [ 1.48726002e+01]]
[[0.54583353]
 [0.02485906]
 [0.03897516]]
[[ 0.05576875]
 [-0.02552299]
 [-0.0209569 ]]
[[-0.25355929]
 [ 0.005749  ]
 [ 0.03889906]]
average registration time:  0.

### Gauss-newton

In [5]:
from planeslam.registration import GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = GN_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

[[30.65781317]
 [11.14045758]
 [24.75049124]]
[[-2.30432372]
 [ 0.41884598]
 [ 2.66917582]]
[[32.10861411]
 [-1.59487608]
 [-2.7139839 ]]
[[ 1.13175403]
 [-1.19555816]
 [-0.70387388]]
[[ 4.46552276]
 [-2.58861795]
 [ 0.43554267]]
[[ 2.57591884]
 [-1.27890639]
 [ 1.22939859]]
[[ 1.01337052]
 [ 4.24488642]
 [-1.43561391]]
[[ 1.88583322]
 [ 4.58631592]
 [-4.81763078]]
[[ 1.66215405]
 [ 0.64699314]
 [-0.33533968]]
[[0.38770594]
 [2.62742646]
 [0.32078439]]
[[0.36142706]
 [2.24109746]
 [1.70968005]]
[[  7.17623909]
 [ 29.53666312]
 [-52.35872499]]
[[8.69197295]
 [0.06868411]
 [2.63284572]]
[[8.61138632]
 [0.45126762]
 [6.60660178]]
[[0.59507523]
 [1.97110034]
 [0.14642334]]
[[0.45874694]
 [1.2864742 ]
 [2.1262965 ]]
[[-0.97621036]
 [ 0.07816455]
 [ 0.05089134]]
[[0.16688146]
 [0.0009609 ]
 [0.02947214]]
[[-0.03333937]
 [ 0.21852243]
 [ 1.64766113]]
average registration time:  0.0013076757129869964


### Torch GN

In [6]:
# Set torch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [30]:
from planeslam.registration import torch_GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = torch_GN_register(scans[i+1], scans[i], device)
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

final loss:  tensor(13.9194, device='cuda:0')
final loss:  tensor(3.5951, device='cuda:0')
final loss:  tensor(3.6772, device='cuda:0')
final loss:  tensor(0.0655, device='cuda:0')
final loss:  tensor(16.9326, device='cuda:0')
final loss:  tensor(9.3114, device='cuda:0')
final loss:  tensor(1.5119, device='cuda:0')
final loss:  tensor(1.5656, device='cuda:0')
final loss:  tensor(1.0765, device='cuda:0')
final loss:  tensor(0.6230, device='cuda:0')
final loss:  tensor(2.5295, device='cuda:0')
final loss:  tensor(0.0006, device='cuda:0')
final loss:  tensor(3370.4016, device='cuda:0')
final loss:  tensor(0.0943, device='cuda:0')
final loss:  tensor(1.2914, device='cuda:0')
final loss:  tensor(0.0487, device='cuda:0')
final loss:  tensor(0.0130, device='cuda:0')
final loss:  tensor(0.0011, device='cuda:0')
final loss:  tensor(0.0401, device='cuda:0')
average registration time:  0.13360580645109477


In [28]:
np.round(abs_traj_transformations[:,:3,3],2)

array([[ 4.02110e+02,  4.63000e+00,  4.61200e+01],
       [-3.22110e+02,  6.28000e+00, -2.29240e+02],
       [-3.05600e+02,  5.54000e+00, -2.13800e+02],
       [-3.03550e+02,  6.37000e+00, -2.11180e+02],
       [-3.01070e+02,  4.11000e+00, -2.09180e+02],
       [-3.01170e+02,  1.83000e+00, -2.08840e+02],
       [-2.93550e+02,  3.15000e+00, -2.13130e+02],
       [-2.86940e+02, -1.16000e+00, -2.15560e+02],
       [-2.83100e+02,  1.99000e+00, -2.14860e+02],
       [-2.75800e+02,  5.04000e+00, -2.18620e+02],
       [-2.84020e+02,  4.82000e+00, -2.01610e+02],
       [-2.85630e+02,  5.57000e+00, -1.74790e+02],
       [ 3.10210e+02,  9.39000e+00, -7.09390e+02],
       [-6.46040e+02,  4.89000e+00, -5.27140e+02],
       [-6.87970e+02,  4.45000e+00, -4.72030e+02],
       [ 1.00957e+03, -3.40060e+02,  2.74723e+03],
       [ 1.12655e+03, -3.33350e+02,  2.70246e+03],
       [ 9.89190e+02, -3.33290e+02,  2.75571e+03],
       [ 9.44690e+02, -5.17030e+02,  2.74291e+03]])

In [12]:
filepath = os.path.join(os.getcwd(), '..', '..', 'data', 'results', 'l2l_abs_traj_transformations')
np.save(filepath, np.array(abs_traj_transformations))

### Torch SGD

In [31]:
from planeslam.registration import torch_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = torch_register(scans[i+1], scans[i], device)
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

Final loss:  13.919158  iterations:  250
Final loss:  4.551803  iterations:  250
Final loss:  3.7154603  iterations:  250
Final loss:  0.012677892  iterations:  250
Final loss:  16.934322  iterations:  250
Final loss:  9.330257  iterations:  250
Final loss:  18.79165  iterations:  250
Final loss:  596.68036  iterations:  250
Final loss:  5.659542  iterations:  250
Final loss:  0.62301403  iterations:  250
Final loss:  0.03943101  iterations:  250
Final loss:  0.0070477733  iterations:  250
Final loss:  0.026667735  iterations:  250
Final loss:  0.14551076  iterations:  250
Final loss:  1.2912554  iterations:  250
Final loss:  0.048118886  iterations:  250
Final loss:  0.0040603126  iterations:  250
Final loss:  0.0010572921  iterations:  250
Final loss:  0.040063262  iterations:  250
average registration time:  1.1333631214342619
